## ECE 285 - Final Project
## Training a neural network to predict image category using functional magnetic resonance imaging (fMRI) signals.
###  Instructor: Xiaolong WANG
*****************************************************************************************************************
Shayne Wang, A15776202, June 2023

ECE MLDS, UCSD

Bochen Pan, A15562542, June 2023

ECE MLDS, UCSD
******************************************************************************************************************

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
#from torch.utils.data import Dataset
#import torchvision
#import torchvision.transforms as transforms
#import torchvision.transforms as T

import numpy as np
import pandas as pd
import os
np.random.seed(0)
#os.chdir('C:/0-ece285/project')

In [18]:
###################################################################################
#  Section 1: DATA IMPORT
#  We import the dataset in .csv format and convert it into a data frame
#  The dataset has been divided into 4 files. 
#  Generate X and Y in numpy from data frame
###################################################################################
pd_fMRI_1 = pd.read_csv('df_merged_subject1.csv')
#pd_fMRI_2 = pd.read_csv('.\data\df_merged_subject2.csv')
#pd_fMRI_3 = pd.read_csv('.\data\df_merged_subject3.csv')
#pd_fMRI_4 = pd.read_csv('.\data\df_merged_subject4.csv')

#pd_fMRI = pd.concat([pd_fMRI_1,pd_fMRI_2,pd_fMRI_3,pd_fMRI_4])

# display the 5 lines of the data frame
print(pd_fMRI_1.head())

# generate X and Y in numpy
X_data = pd_fMRI_1.values[:,2:-1].astype(float)
Y_data = pd_fMRI_1.values[:,-1]
print('shape of data', X_data.shape, Y_data.shape)

# filter out the name of categories
Y_class = set(Y_data.tolist())
# create a dictionary for category vs index
class_dict = dict(zip(Y_class,range(len(Y_class))))
print(class_dict)

# convert category Y into integer since PyTorch doesn't have dtype for string
Y_data = np.array([class_dict.get(a_class) for a_class in Y_data.tolist()])


   subject                         image_id  LHEarlyVis#0  LHEarlyVis#1  \
0        1             n01930112_19568.JPEG      0.167271      1.173377   
1        1             n03733281_29214.JPEG      1.258984      0.744704   
2        1              n07695742_5848.JPEG      0.279666     -0.164892   
3        1  COCO_train2014_000000420713.jpg      0.473376     -0.299339   
4        1  COCO_train2014_000000488558.jpg      0.224416      1.852141   

   LHEarlyVis#2  LHEarlyVis#3  LHEarlyVis#4  LHEarlyVis#5  LHEarlyVis#6  \
0     -0.482830      0.561836      0.487629     -1.366299      0.526726   
1      0.264117     -0.199035      0.221795     -1.114712      0.549931   
2     -0.550474      0.587374      0.319142     -0.022280      1.146169   
3      0.365422      0.443424      0.986940      0.916352      0.656573   
4      1.087473     -0.393302      0.129446      0.858346     -0.352569   

   LHEarlyVis#7  ...  RHRSC#134  RHRSC#135  RHRSC#136  RHRSC#137  RHRSC#138  \
0      0.645983  ..

In [19]:
###################################################################################
#  Section 2: DATA PREPROCESSING AND SPLIT
#  Preprocess the data in this section
#  Split the dataset into validation : test = 80% : 20% . 
###################################################################################
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import TensorDataset

# Option to use GPU
#USE_GPU = False  # Dad can not use GPU
USE_GPU = True
num_class = len(class_dict)
dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

Tensor = torch.cuda.FloatTensor if USE_GPU else torch.FloatTensor

batch_size = 32

# convert numpy arrays to tensors, then encapsulate as a dataset (X,Y) ready to split
dataset = TensorDataset(Tensor(X_data), Tensor(Y_data))
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Constant to control how frequently we print train loss
print_every = 100

using device: cuda


# Model 1 neural network without dropout

In [9]:
# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Initialize the model, loss function and optimizer
input_size = X_data.shape[1]  # number of features in the input
hidden_size = 100  # You can change this value
output_size = len(class_dict)  # number of classes in the output

model = NeuralNetwork(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can change the learning rate

# Train the model
num_epochs = 10  # You can change the number of epochs
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.long())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        if (i+1) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_dataloader), loss.item()))


Epoch [1/10], Step [100/126], Loss: 0.9962
Epoch [2/10], Step [100/126], Loss: 0.7680
Epoch [3/10], Step [100/126], Loss: 0.5321
Epoch [4/10], Step [100/126], Loss: 0.3983
Epoch [5/10], Step [100/126], Loss: 0.0864
Epoch [6/10], Step [100/126], Loss: 0.0697
Epoch [7/10], Step [100/126], Loss: 0.0249
Epoch [8/10], Step [100/126], Loss: 0.0167
Epoch [9/10], Step [100/126], Loss: 0.0105
Epoch [10/10], Step [100/126], Loss: 0.0080


In [20]:
# Test the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Turn off gradients for this block
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 79.96031746031746 %


# Model with Dropout and Batch Normalization

In [21]:

# Define the model with Dropout and Batch Normalization
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.5):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Batch Normalization after first layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)  # Dropout layer
        self.fc2 = nn.Linear(hidden_size, output_size)  

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

# Initialize the model, loss function and optimizer
model = NeuralNetwork(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler for learning rate decay
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Early stopping initialization
n_epochs_stop = 5
min_val_loss = np.Inf
epochs_no_improve = 0

# Train the model
for epoch in range(num_epochs):
    # Training
    model.train()
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.long())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        if (i+1) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_dataloader), loss.item()))

    # Validation
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        val_loss = 0
        for images, labels in valid_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels.long())
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        val_loss = val_loss/len(valid_dataloader)

        print('Validation accuracy is: {} %, loss is: {}'.format(100 * correct / total, val_loss))

    # Early stopping
    if min_val_loss > val_loss:
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!')
            break

    # Step the scheduler
    scheduler.step()


Epoch [1/10], Step [100/126], Loss: 1.3221
Validation accuracy is: 52.788844621513945 %, loss is: 1.1903143301606178
Epoch [2/10], Step [100/126], Loss: 1.0487
Validation accuracy is: 55.57768924302789 %, loss is: 1.1702259480953217
Epoch [3/10], Step [100/126], Loss: 0.7791
Validation accuracy is: 53.386454183266935 %, loss is: 1.1971449740231037
Epoch [4/10], Step [100/126], Loss: 0.9117
Validation accuracy is: 51.79282868525896 %, loss is: 1.231712929904461
Epoch [5/10], Step [100/126], Loss: 0.6955
Validation accuracy is: 52.191235059760956 %, loss is: 1.2543550245463848
Epoch [6/10], Step [100/126], Loss: 0.6235
Validation accuracy is: 53.386454183266935 %, loss is: 1.310781192034483
Epoch [7/10], Step [100/126], Loss: 0.6007
Validation accuracy is: 50.59760956175299 %, loss is: 1.3456846699118614
Early stopping!


In [22]:
# Test the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Turn off gradients for this block
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 58.73015873015873 %


# Model with Dropout and Batch Normalization with more complexity and more layers

In [23]:

# Define the model with Dropout and Batch Normalization with more complexity and more layers
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, output_size, dropout_prob=0.5):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Batch Normalization after first layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)  # Dropout layer

        self.fc2 = nn.Linear(hidden_size, hidden_size2)  # Second hidden layer
        self.bn2 = nn.BatchNorm1d(hidden_size2)  # Batch Normalization after second layer

        self.fc3 = nn.Linear(hidden_size2, output_size)  

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc3(out)
        return out

# Initialize the model, loss function and optimizer
hidden_size2 = 200  # Size of the second hidden layer, you can change this value
model = NeuralNetwork(input_size, hidden_size, hidden_size2, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler for learning rate decay
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Early stopping initialization
n_epochs_stop = 5
min_val_loss = np.Inf
epochs_no_improve = 0

# Train the model
for epoch in range(num_epochs):
    # Training
    model.train()
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.long())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        if (i+1) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_dataloader), loss.item()))

    # Validation
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        val_loss = 0
        for images, labels in valid_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels.long())
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        val_loss = val_loss/len(valid_dataloader)

        print('Validation accuracy is: {} %, loss is: {}'.format(100 * correct / total, val_loss))

    # Early stopping
    if min_val_loss > val_loss:
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!')
            break

    # Step the scheduler
    scheduler.step()


Epoch [1/10], Step [100/126], Loss: 1.4661
Validation accuracy is: 52.39043824701195 %, loss is: 1.2184390909969807
Epoch [2/10], Step [100/126], Loss: 1.1219
Validation accuracy is: 54.581673306772906 %, loss is: 1.1902594044804573
Epoch [3/10], Step [100/126], Loss: 1.4515
Validation accuracy is: 56.77290836653386 %, loss is: 1.1652898080646992
Epoch [4/10], Step [100/126], Loss: 1.3032
Validation accuracy is: 58.167330677290835 %, loss is: 1.162667192518711
Epoch [5/10], Step [100/126], Loss: 0.8456
Validation accuracy is: 57.76892430278885 %, loss is: 1.1601295173168182
Epoch [6/10], Step [100/126], Loss: 1.0854
Validation accuracy is: 56.17529880478088 %, loss is: 1.166124615818262
Epoch [7/10], Step [100/126], Loss: 0.6936
Validation accuracy is: 54.38247011952191 %, loss is: 1.2391556948423386
Epoch [8/10], Step [100/126], Loss: 0.4609
Validation accuracy is: 55.57768924302789 %, loss is: 1.2728989943861961
Epoch [9/10], Step [100/126], Loss: 0.8803
Validation accuracy is: 54.58

In [24]:
# Evaluate the model
model.eval()  # it's important to switch to eval mode for testing
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 49.007936507936506 %


In [25]:
# above overfitting

In [ ]:
# use dataloader to feed the model
def train():
    




In [ ]:
def evaluate():

7